In [0]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from utils.ingestUtility import *
import yfinance as yf
import pandas as pd

In [0]:
symbols = [
    "AAPL", "META", "TSLA", "GOOG", "NFLX",
    "GOOGL", "WMT", "AMD", "AMZN", "MSFT",
    "NVDA", "DIS", "KO", "PLTR"
]

all_data = []

for symbol in symbols:
    data = yf.download(symbol, period='1d')
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.droplevel(0)
    data.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
    data.reset_index(inplace=True)
    data = data[['Date', 'Close', 'High', 'Low', 'Open', 'Volume']]
    data['Ticker'] = symbol
    all_data.append(data)

final_df = pd.concat(all_data, ignore_index=True)

print(final_df)

In [0]:
df = spark.createDataFrame(final_df)
df = df.withColumn("date", F.to_date(F.col("Date"), "yyyy-MM-dd"))
display(df)

In [0]:
df.write.format("delta").mode("append").saveAsTable("pprtrading.stocks_data")